In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

/home/sven/dev/DHBW-KI-Laborarbeit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
file_path = '../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')

# Bewohnerkategorie bestimmen
resident_columns = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[resident_columns].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ['Hausmeister', 'Garage', 'Aufzug', 'Balkon', 'Terrasse', "Kehrwoche", "Studierende", "Kleinfamilie", "DINK", "Alleinerziehende", "Expatriate", "Rentnerpaar"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

# Stockwerk
def get_stockwerk(range_str):
    if isinstance(range_str, str):
        if "EG" in range_str:
            return 0
        elif "1.Stock" in range_str:
            return 1
        elif "2.Stock" in range_str:
            return 2
        elif "3.Stock" in range_str:
            return 3
        elif "4.Stock" in range_str:
            return 4
        elif "5.Stock" in range_str:
            return 5
        elif "6.Stock" in range_str:
            return 6
        elif "7.Stock" in range_str:
            return 7
        elif "8.Stock" in range_str:
            return 8
        elif "9.Stock" in range_str:
            return 9
    return range_str

data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)

# Heizung
def get_heizung(range_str):
    if isinstance(range_str, str):
        if "Erdwaerme" in range_str:
            return 0
        elif "Gas-Etage" in range_str:
            return 1
        elif "Gas-Zentral" in range_str:
            return 2
        elif "Lüftungsheizung" in range_str:
            return 3
        elif "Oel" in range_str:
            return 4
        elif "Pellets" in range_str:
            return 5
    return range_str

data['Heizung'] = data['Heizung'].apply(get_heizung)

# Funktion zum Berechnen des Durchschnitts aus einem Wertebereich
def calculate_average_range_nebenkosten(range_str):
    if isinstance(range_str, str):
        if "ueber" in range_str:
            return 300
        elif "unter" in range_str:
            return 50
        else:
            # Teile den Wertebereich in zwei Zahlen auf
            start, end = map(int, range_str.split('-'))
            # Berechne den Durchschnitt
            return (start + end) / 2
    return range_str

# Wende die Funktion auf die Spalte "Quadratmeter" an
data['Nebenkosten'] = data['Nebenkosten'].apply(calculate_average_range_nebenkosten)

# Alter
def calculate_average_range_alter(range_str):
    if isinstance(range_str, str):
        if "ueber" in range_str:  # "ueber 100 Jahre"
            return 100
        elif "Neubau" in range_str:  # Neubau als 0 interpretieren
            return 0
        else:
            # Splitte den Wertebereich bei '-' und entferne "Jahre"
            numbers = range_str.split('-')
            start, end = list(map(int, [numbers[0].strip(), numbers[1].replace("Jahre", "").strip()]))
            return (start + end) / 2  # Durchschnitt berechnen
    return range_str

data['Alter'] = data['Alter'].apply(calculate_average_range_alter)

# Funktion zum Berechnen des Durchschnitts aus einem Wertebereich
def calculate_average_range(range_str):
    if isinstance(range_str, str):
        if "ueber" in range_str:  # z.B. "ueber 100 Jahre"
            return 100
        elif "Neubau" in range_str:  # Neubau als 0 interpretieren
            return 0
        else:
            # Splitte den Wertebereich bei '-' und entferne "Jahre"
            numbers = range_str.split('-')
            start = int(numbers[0].strip())  # Erster Wert
            end = int(numbers[1].replace("Jahre", "").strip())  # "Jahre" entfernen und bereinigen
            return (start + end) / 2  # Durchschnitt berechnen

    return range_str


# Wende die Funktion auf die Spalte "Quadratmeter" an
data['Quadratmeter'] = data['Quadratmeter'].apply(calculate_average_range)

# Lage
def get_lage(range_str):
    if isinstance(range_str, str):
        if "Abgelegen" in range_str:
            return 0
        elif "Wohngebiet" in range_str:
            return 0
        elif "Hauptstrasse" in range_str:
            return 1
        elif "Nebenstrasse" in range_str:
            return 2
        elif "Spielstrasse" in range_str:
            return 3
    return range_str

data['Lage'] = data['Lage'].apply(get_lage)

# Kaution
def calculate_average_range_kaution(range_str):
    if isinstance(range_str, str):
        if "ueber" in range_str:  # "ueber 100 Jahre"
            return 3000
        elif "keine" in range_str:  # Neubau als 0 interpretieren
            return 0
        else:
            start, end = map(int, range_str.split('-'))
            return (start + end) / 2
    return range_str

data['Kaution'] = data['Kaution'].apply(calculate_average_range_kaution)

# Kueche
def get_kueche(range_str):
    if isinstance(range_str, str):
        if "keine" in range_str:
            return 0
        elif "Kueche (alt)" in range_str:
            return 1
        elif "Kueche (neu)" in range_str:
            return 2
    return range_str

data['Kueche'] = data['Kueche'].apply(get_kueche)

# Kueche
def get_bad(range_str):
    if isinstance(range_str, str):
        if "Badewanne" in range_str:
            return 0
        elif "Dusche" in range_str:
            return 1
        elif "Dusche + Wanne" in range_str:
            return 2
        elif "Dusche auf Flur" in range_str:
            return 3
        elif "Waschbecken" in range_str:
            return 4
    return range_str

data['Bad'] = data['Bad'].apply(get_bad)

# Kindergarten
def get_kindergarten(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 2
        elif "erreichbar" in range_str:
            return 1
        elif "fern" in range_str:
            return 0
    return range_str

data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)

# Schule
def get_schule(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 2
        elif "erreichbar" in range_str:
            return 1
        elif "fern" in range_str:
            return 0
    return range

data['Schule'] = data['Schule'].apply(get_schule)

# S-Bahn
def get_bahn(range_str):
    if isinstance(range_str, str):
        if "nah" in range_str:
            return 4
        elif "erreichbar" in range_str:
            return 3
        elif "mit Bus" in range_str:
            return 2
        elif "fern" in range_str:
            return 1
        elif "nein" in range_str:
            return 0
    return range

data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)

# miete
def calculate_miete(value):
    if '-' in value:
        start, end = value.split('-')
        return (int(start) + int(end)) / 2
    else:
        return int(value)

data['Miete'] = data['Miete'].apply(calculate_miete)

# Zimmer
def kategorisiere_zimmer(zimmer):
    if zimmer in ['Ein Zimmer', '1 Zimmer']:
        return 0
    elif zimmer in ['Zwei Zimmer', 'Drei Zimmer', '2 Zimmer', '3 Zimmer', '1-2 Zimmer', '2-3 Zimmer', ]:
        return 1
    elif zimmer in ['Vier Zimmer', 'Fünf Zimmer', '3-4 Zimmer', '4-5 Zimmer', '4 Zimmer', '5 Zimmer']:
        return 2
    elif zimmer in ['Sechs Zimmer', '5-6 Zimmer', '6 Zimmer']:
        return 3

data['Zimmerzahl'] = data['Zimmerzahl'].apply(kategorisiere_zimmer)

# mobliert
def get_mobliert(range_str):
    if isinstance(range_str, str):
        if "ja" in range_str:
            return 2
        elif "teilmoebliert" in range_str:
            return 1
        elif "nein" in range_str:
            return 0
    return range

data['Moebliert'] = data['Moebliert'].apply(get_mobliert)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0  725.5        275.5  ...          0          0         115.5   
13       1  725.5        225.5  ...          0          0         105.5   
23       0  775.5        275.5  ...          0          0         115.5   
27       0  225.5         50.0  ...          0          0          25.0   
40       0  225.5         50.0  ...          0          0          35.5   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

/tmp/ipykernel_34093/3607250715.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [37]:
# Modellierung des Bayes-Netzes
resident_columns = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']

model = BayesianNetwork([
    ('Zimmerzahl', 'Bewohnerkategorie'),
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Heizung', 'Bewohnerkategorie'),
    ('Hausmeister', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('Schule', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie'),
    ('Garage', 'Bewohnerkategorie'),
    ('Miete', 'Bewohnerkategorie'),
    ('Nebenkosten', 'Bewohnerkategorie'),
    ('Alter', 'Bewohnerkategorie'),
    ('Aufzug', 'Bewohnerkategorie'),
    ('Lage', 'Bewohnerkategorie'),
    ('Kaution', 'Bewohnerkategorie'),
    ('Kueche', 'Bewohnerkategorie'),
    ('Bad', 'Bewohnerkategorie'),
    ('Balkon', 'Bewohnerkategorie'),
    ('Terrasse', 'Bewohnerkategorie'),
    ('Kehrwoche', 'Bewohnerkategorie'),
    ('Moebliert', 'Bewohnerkategorie'),
    ('Quadratmeter', 'Bewohnerkategorie')
])

# Schritt 4: Training des Bayes-Netzes
model.fit(data, estimator=MaximumLikelihoodEstimator)


# Schritt 5: Inferenz durchführen
inference = VariableElimination(model)

# Beispiel-Eingabe: Teilweise bekannte Daten
example_input = {
    "Zimmerzahl": 1,  # Verwende korrekte Zustandsnummer
    "Miete": 0,           # Verwende korrekte Zustandsnummer
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.map_query(variables=["Alleinerziehende", "Expatriate", "Rentnerpaar", "Studierende", "DINK", "Kleinfamilie"], evidence=example_input)
'''
print("Klassifikationsergebnis:")
print(prediction)
'''

MemoryError: Unable to allocate 833. GiB for an array with shape (893931945984,) and data type int8